In [1]:
!pip install torch transformers sentence-transformers faiss-cpu gradio PyMuPDF accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.8 MB/s eta 0:00:00


Here's an example of how you can customize the Gradio interface using themes and custom CSS.

In [9]:
!pip install gradio transformers accelerate safetensors pymupdf python-docx -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.1 MB/s eta 0:00:00


You can experiment with different themes and CSS styles to achieve the desired look.

Once you share the console errors, I can help you troubleshoot why the application is not displaying output.

In [11]:
import gradio as gr
import fitz  # PyMuPDF for PDFs
import docx  # python-docx for Word files
from transformers import pipeline

# ---------------- Load Granite Model ----------------
qa_pipeline = pipeline(
    "text-generation",
    model="ibm-granite/granite-3.3-2b-instruct", # Corrected model name
    device_map="auto"
)

# ---------------- Helpers ----------------
def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def chunk_text(text, chunk_size=1500, overlap=200):
    """Split text into overlapping chunks to avoid cutting sentences."""
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# ---------------- Main Function ----------------
def answer_from_pdf(file, question):
    if file is None:
        return "⚠️ No file uploaded.", "N/A"

    try:
        if file.name.endswith(".pdf"):
            text = extract_text_from_pdf(file.name)
        elif file.name.endswith(".docx"):
            text = extract_text_from_docx(file.name)
        else:
            return "❌ Unsupported file format. Please upload PDF or DOCX.", "N/A"
    except Exception as e:
        return f"⚠️ Error reading file: {str(e)}", "N/A"

    # Split document into chunks
    chunks = chunk_text(text, chunk_size=1500, overlap=200)

    responses = []
    try:
        if "summary" in question.lower():
            # Summarize each chunk
            for chunk in chunks:
                prompt = f"Summarize this part of study material:\n\n{chunk}"
                result = qa_pipeline(prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]
                responses.append(result)

            # Merge into one overall summary
            final_prompt = "Combine these summaries into one clear and concise summary:\n\n" + "\n".join(responses)
            final_answer = qa_pipeline(final_prompt, max_new_tokens=400, do_sample=False)[0]["generated_text"]
            return final_answer, "📘 Granite (chunked summarization)"
        else:
            # Answer Q&A from chunks
            for chunk in chunks[:5]:  # limit to first 5 chunks for speed
                prompt = f"Context:\n{chunk}\n\nQuestion: {question}\nAnswer clearly."
                result = qa_pipeline(prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]
                responses.append(result)

            final_answer = "\n\n".join(responses)
            return final_answer, f"💡 Granite (chunked Q&A, used {len(chunks[:5])} chunks)"
    except Exception as e:
        return f"⚠️ Error from Granite model: {str(e)}", "N/A"

# ---------------- Beautiful UI ----------------
custom_css = """
body {
  background: linear-gradient(135deg, #f0f9ff, #cbebff);
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
#title {
  text-align: center;
  font-size: 2.2em;
  color: #1f3b73;
  margin-bottom: 10px;
}
#desc {
  text-align: center;
  font-size: 1.1em;
  margin-bottom: 20px;
  color: #333;
}
.gradio-container {
  max-width: 950px !important;
  margin: auto;
  border-radius: 20px;
  padding: 20px;
  box-shadow: 0 8px 25px rgba(0,0,0,0.15);
  background: white;
}
textarea {
  border-radius: 12px !important;
  padding: 12px !important;
  font-size: 1em !important;
}
input[type=file] {
  border-radius: 12px !important;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as iface:
    with gr.Column():
        gr.HTML("<h1 id='title'>📚 StudyMate – AI Academic Assistant</h1>")
        gr.HTML("<p id='desc'>Upload your study materials (PDF/DOCX) and ask questions. Powered by <b>IBM Granite (Hugging Face)</b>.</p>")

        with gr.Row():
            file_input = gr.File(label="📂 Upload PDF or DOCX", file_types=[".pdf", ".docx"])

        question_input = gr.Textbox(
            lines=2,
            placeholder="Type your question here (e.g., 'Give me a summary')",
            label="❓ Your Question"
        )

        with gr.Row():
            answer_output = gr.Textbox(label="📝 Answer", lines=10)
            context_output = gr.Textbox(label="📖 Context Used", lines=2)

        submit_btn = gr.Button("🚀 Get Answer", variant="primary")

        submit_btn.click(
            fn=answer_from_pdf,
            inputs=[file_input, question_input],
            outputs=[answer_output, context_output]
        )

iface.launch(share=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4cd9ee78161ae60cfa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
